In [1]:
%load_ext cython

In [2]:
import numpy as np
import pywt

In [56]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from pywt._extensions import _dwt
from pywt._extensions._pywt import Modes
from pywt._utils import _as_wavelet
from numpy import zeros, ceil, sum, intc, double as ndouble
from libc.math cimport log2


cdef void _dwt_single(const double[:] x, int nc, double[:] cA, double[:] cD):
    wave = _as_wavelet('coif4')
    mode = Modes.from_object('symmetric')
    
    cdef double[:] cA_, cD_
    cA_, cD_ = _dwt.dwt_single(x, wave, mode)
    
    cA[:nc] = cA_
    cD[:] = cD_
    

cpdef cdetail_power(const double[:] x, double fs, double[:] f_band):
    # parameters
    mode = Modes.from_object('symmetric')
    wave = _as_wavelet('coif4')
    
    # determine the levels we need
    cdef int[:] levels = zeros(2, dtype=intc)
    levels[0] = <int>ceil(log2(fs / f_band[0]))
    levels[1] = <int>ceil(log2(fs / f_band[1]))
    
    if levels[0] > 10:
        raise ValueError('Deconstruction over 10 levels not supported')
    
    # determine the maximum coefficient length
    cdef int nwave = len(wave.filter_bank[0])
    cdef int nprev, n = _dwt.dwt_coeff_len(x.size, nwave, mode)
    
    # allocate space for the approximation coefficients
    cdef double[:] cA = zeros(x.size, dtype=ndouble)
    
    # save all the coefficients
    cdef int[:] n_coefs = zeros(levels[0] + 1, dtype=intc)
    n_coefs[0] = x.size
    n_coefs[1] = n
    
    cdef Py_ssize_t i, lvl
    for lvl in range(levels[0]-1, 0, -1):
        i = levels[0] - lvl
        n_coefs[i+1] = _dwt.dwt_coeff_len(n_coefs[i], nwave, mode)
    
    # space for the detail coefficients
    cdef double[:] cD = zeros(sum(n_coefs) - x.size, dtype=ndouble)
    cA[:] = x
    
    for i in range(1, levels[0]):
        nprev = sum(n_coefs[:i-1])
        n = sum(n_coefs[1:i])  # repurpose
#         cA[:n_coefs[i]], cD[n:n + n_coefs[i]] = _dwt.dwt_single(cD[nprev:nprev + n_coefs[i-1]], wave, mode)
        _dwt_single(cA[:n_coefs[i-1]], n_coefs[i], cA[:n_coefs[i]], cD[n:n + n_coefs[i]])
    
    return cA, cD

In [7]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
import pywt
from numpy import zeros, diff, sum, sign, ceil, intc, power
from libc.math cimport log2

cpdef cy_power(const double[::1] x, double fs, double[:] f_band):
    cdef int[:] levels = zeros(2, dtype=intc)
    levels[0] = <int>ceil(log2(fs / f_band[0]))
    levels[1] = <int>ceil(log2(fs / f_band[1]))
    
    cA, *cD = pywt.wavedec(x, 'coif4', mode='symmetric', level=levels[0])
    
    for i in range(levels[0] - levels[1] + 1, levels[0]):
        cD[i][:] = 0.0
    
    xr = pywt.waverec((cA,) + tuple(cD), 'coif4', mode='symmetric')
    N = sum(diff(sign(xr)) > 0)
    if N == 0:
        N = 0.001  # prevent divide by 0
    
    cdef double pwr = 0.0
    for i in range(levels[0] - levels[1] + 1):
        pwr += sum(power(cD[i], 2.0))
    
    pwr /= N
    return pwr

In [59]:
def detail_power(x, fs, wave='coif4', f_band=[1, 3], levels=None):
    if levels is None and f_band is not None:
        f_band = np.sort(f_band)
        levels = [
            int(np.ceil(np.log2(fs / f_band[0]))),  # maximum level we need
            int(np.ceil(np.log2(fs / f_band[1])))  # minimum level to include in sum
        ]
    
    # TODO test effect of mode on the result
    cA, *cD = pywt.wavedec(x, wave, mode='symmetric', level=levels[0])
    
    for i in range(levels[0] - levels[1] + 1, levels[0]):
        cD[i][:] = 0.
    
    xr = pywt.waverec((cA,) + tuple(cD), wave, mode='symmetric')
    N = np.sum(np.diff(np.sign(xr)) > 0)  # get the number of positive to negative crossings
    if N == 0:
        N = 0.001  # prevent divide by 0
    
    power = 0
    for i in range(levels[0] - levels[1] + 1):
        power += np.sum(cD[i]**2)
    power /= N
    
    return power
      

def detail_power_ratio(x, fs, wave='coif4', f_band=[1, 10], levels=None):
    if levels is None and f_band is not None:
        f_band = np.sort(f_band)
        levels = [
            int(np.ceil(np.log2(fs / f_band[0]))),  # maximum level we need
            int(np.ceil(np.log2(fs / f_band[1])))  # minimum level to include in sum
        ]
    
    # TODO test effect of mode on the result
    cA, *cD = pywt.wavedec(x, wave, mode='symmetric', level=levels[0])
    
    power = 0
    for i in range(levels[0] - levels[1] + 1):
        power += np.sum(cD[i]**2)
    
    power /= np.sum(x**2)
    
    return power * 100

In [9]:
x = np.random.rand(150)

In [11]:
detail_power(x, 50.0, f_band=[1, 3])

1306.437577976551

In [10]:
cy_power(x, 50.0, np.array([1.0, 3.0]))

/Users/adamol/opt/miniconda3/envs/pfymu/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(


1306.437577976551

In [60]:
%timeit detail_power(x, 50, wave='coif4', f_band=[1, 3])

278 µs ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit detail_power_ratio(x, 50.0, wave='coif4', levels=[6, 3])

In [ ]:
detail_power_ratio(x, 50.0, levels=[6, 3])

In [ ]:
len(cD)

xr = (
    pywt.upcoef('a', cA, 'coif4', level=6, take=150) + 
    pywt.upcoef('d', cD[0], 'coif4', level=6, take=150) + 
    pywt.upcoef('d', cD[1], 'coif4', level=5, take=150) +
    pywt.upcoef('d', cD[2], 'coif4', level=4, take=150) +
    pywt.upcoef('d', cD[3], 'coif4', level=3, take=150) +
    pywt.upcoef('d', cD[4], 'coif4', level=2, take=150) +
    pywt.upcoef('d', cD[5], 'coif4', level=1, take=150)
)

In [ ]:
type(cD)

In [ ]:
x = np.random.rand(150)

pywt.dwt_max_level(600, 'coif4')

In [ ]:
np.log2(150)

In [16]:
cA, *cD = pywt.wavedec(x, 'coif4', level=6)

/Users/adamol/opt/miniconda3/envs/pfymu/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
for i in range(10, 0, -1):
    print(f'{i:2d}{50 / 2**i:7.2f}-{50 / 2**(i-1):7.2f}')